In [78]:
import os
import pandas as pd 
import matplotlib as plt
from wordcloud import WordCloud
import seaborn as sns
import re
#import pickle
import numpy as np
import csv
#import string
#import nltk
#import warnings


for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/twitter-dataset/train_neg.txt
/kaggle/input/twitter-dataset/train_pos.txt
/kaggle/input/twitter-dataset/test_data.txt
/kaggle/input/twitter-dataset/train_neg_full.txt
/kaggle/input/twitter-dataset/train_pos_full.txt


In [79]:
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import BernoulliNB
from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import confusion_matrix, classification_report

In [80]:
file1 = '/kaggle/input/twitter-dataset/train_pos.txt'
file2 = '/kaggle/input/twitter-dataset/train_neg.txt'

df = pd.read_fwf(file1)
df.to_csv('train_pos.csv', index=False)

df = pd.read_fwf(file2)
df.to_csv('train_neg.csv', index=False)

In [81]:
import csv
with open('train_neg.csv', 'r') as file:
        # Read the existing data from the CSV file
        reader = csv.reader(file)
        data = list(reader)

    # Insert column with 1s at the beginning of each row
for row in data:
        row.insert(0, '1')

with open('train_neg_merged.csv', 'w', newline='') as file:
        # Write the modified data back to the CSV file
        writer = csv.writer(file)
        writer.writerows(data)
        
        
with open('train_pos.csv', 'r') as file:
        # Read the existing data from the CSV file
        reader = csv.reader(file)
        data = list(reader)

    # Insert column with 1s at the beginning of each row
for row in data:
        row.insert(0, '0')

with open('train_pos_merged.csv', 'w', newline='') as file:
        # Write the modified data back to the CSV file
        writer = csv.writer(file)
        writer.writerows(data)

In [82]:

# Process negative tweets
with open('train_neg.csv', 'r') as file:
    reader = csv.reader(file)
    data = list(reader)

# Add headers for 'id' and 'label'
data.insert(0, ['id', 'label', 'tweet'])

# Insert 'id' and 'label' for each row
for i, row in enumerate(data[1:], start=1):  # Start at 1 to skip header
    row.insert(0, '-1')  # Insert label at the beginning of the row
    row.insert(0, str(i))  # Insert ID at the beginning of the row

# Write to a new CSV file with headers
with open('train_neg_merged.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerows(data)

# Process positive tweets
with open('train_pos.csv', 'r') as file:
    reader = csv.reader(file)
    data = list(reader)

# Add headers for 'id' and 'label'
data.insert(0, ['id', 'label', 'tweet'])  # Same assumption as above

# Insert 'id' and 'label' for each row
for i, row in enumerate(data[1:], start=1):  # Start at 1 to skip header
    row.insert(0, '1')  # Insert label at the beginning of the row
    row.insert(0, str(i))  # Insert ID at the beginning of the row

# Write to a new CSV file with headers
with open('train_pos_merged.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerows(data)

In [83]:
first_row = [row for row in open('train_neg_merged.csv')][3]
print(first_row)

3,-1,1-3 vs celtics in the regular season = were fucked if we play them in the playoffs



In [84]:
# Load the CSV files
df1 = pd.read_csv('train_neg_merged.csv')
df2 = pd.read_csv('train_pos_merged.csv')

print(df1.shape)
print(df2.shape)

# Combine the DataFrames
combined_tweets = pd.concat([df1, df2], ignore_index=True)

# Save the combined DataFrame to a CSV file
combined_tweets.to_csv('train_data.csv', index=False)
train  = pd.read_csv('train_data.csv')
train[train['label'] == -1].head(10)

(100000, 3)
(100000, 3)


,id,label,tweet
0,1,-1,vinco tresorpack 6 ( difficulty 10 of 10 objec...
1,2,-1,glad i dot have taks tomorrow ! ! #thankful #s...
2,3,-1,1-3 vs celtics in the regular season = were fu...
3,4,-1,<user> i could actually kill that girl i'm so ...
4,5,-1,<user> <user> <user> i find that very hard to ...
5,6,-1,wish i could be out all night tonight ! <user>
6,7,-1,<user> i got kicked out the wgm
7,8,-1,rt <user> <user> <user> yes she is ! u tell it...
8,9,-1,why is she so perfect <url>
9,10,-1,<user> hi harry ! did u havea good time in aus...


In [85]:
train.shape

(200000, 3)

In [86]:
train["label"].value_counts()

label
-1    100000
 1    100000
Name: count, dtype: int64

In [87]:
with open('/kaggle/input/twitter-dataset/test_data.txt', 'r') as f:
    lines = f.readlines()

with open('test.csv', 'w', newline='', encoding='utf-8') as csvfile:
    writer = csv.writer(csvfile, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    writer.writerow(['id', 'tweet'])  # Write the header row

    for line in lines:
        tweet_id, tweet_text = line.split(',', 1)
        writer.writerow([tweet_id, tweet_text.strip()])

In [88]:
test=pd.read_csv('test.csv')
test.shape

(10000, 2)

In [90]:
combi = train._append(test, ignore_index=True)
combi.shape

(210000, 3)

In [91]:
def drop_fragment(input_tweets, to_replace):
    for match in re.findall(pattern, tweets):
        input_tweets = input_tweets.replace(match, '')
    return input_tweets    

In [92]:
combi['clean_tweet'] = combi['tweet'].str.replace("<user>", "")
combi.head()

,id,label,tweet,clean_tweet
0,1,-1.0,vinco tresorpack 6 ( difficulty 10 of 10 objec...,vinco tresorpack 6 ( difficulty 10 of 10 objec...
1,2,-1.0,glad i dot have taks tomorrow ! ! #thankful #s...,glad i dot have taks tomorrow ! ! #thankful #s...
2,3,-1.0,1-3 vs celtics in the regular season = were fu...,1-3 vs celtics in the regular season = were fu...
3,4,-1.0,<user> i could actually kill that girl i'm so ...,i could actually kill that girl i'm so sorry ...
4,5,-1.0,<user> <user> <user> i find that very hard to ...,i find that very hard to believe im afraid


In [93]:
combi['clean_tweet'] = combi['clean_tweet'].str.replace("[^a-zA-Z#]", " ")
combi.head(10)

,id,label,tweet,clean_tweet
0,1,-1.0,vinco tresorpack 6 ( difficulty 10 of 10 objec...,vinco tresorpack 6 ( difficulty 10 of 10 objec...
1,2,-1.0,glad i dot have taks tomorrow ! ! #thankful #s...,glad i dot have taks tomorrow ! ! #thankful #s...
2,3,-1.0,1-3 vs celtics in the regular season = were fu...,1-3 vs celtics in the regular season = were fu...
3,4,-1.0,<user> i could actually kill that girl i'm so ...,i could actually kill that girl i'm so sorry ...
4,5,-1.0,<user> <user> <user> i find that very hard to ...,i find that very hard to believe im afraid
5,6,-1.0,wish i could be out all night tonight ! <user>,wish i could be out all night tonight !
6,7,-1.0,<user> i got kicked out the wgm,i got kicked out the wgm
7,8,-1.0,rt <user> <user> <user> yes she is ! u tell it...,rt yes she is ! u tell it ! my lips are clo...
8,9,-1.0,why is she so perfect <url>,why is she so perfect <url>
9,10,-1.0,<user> hi harry ! did u havea good time in aus...,hi harry ! did u havea good time in aus ? i d...


In [94]:
combi['clean_tweet'] = combi['clean_tweet'].str.replace("<url>", "")
combi.head(10)

,id,label,tweet,clean_tweet
0,1,-1.0,vinco tresorpack 6 ( difficulty 10 of 10 objec...,vinco tresorpack 6 ( difficulty 10 of 10 objec...
1,2,-1.0,glad i dot have taks tomorrow ! ! #thankful #s...,glad i dot have taks tomorrow ! ! #thankful #s...
2,3,-1.0,1-3 vs celtics in the regular season = were fu...,1-3 vs celtics in the regular season = were fu...
3,4,-1.0,<user> i could actually kill that girl i'm so ...,i could actually kill that girl i'm so sorry ...
4,5,-1.0,<user> <user> <user> i find that very hard to ...,i find that very hard to believe im afraid
5,6,-1.0,wish i could be out all night tonight ! <user>,wish i could be out all night tonight !
6,7,-1.0,<user> i got kicked out the wgm,i got kicked out the wgm
7,8,-1.0,rt <user> <user> <user> yes she is ! u tell it...,rt yes she is ! u tell it ! my lips are clo...
8,9,-1.0,why is she so perfect <url>,why is she so perfect
9,10,-1.0,<user> hi harry ! did u havea good time in aus...,hi harry ! did u havea good time in aus ? i d...


In [96]:
combi['clean_tweet'] = combi['clean_tweet'].str.strip()
combi.head(10)

,id,label,tweet,clean_tweet
0,1,-1.0,vinco tresorpack 6 ( difficulty 10 of 10 objec...,vinco tresorpack 6 ( difficulty 10 10 object :...
1,2,-1.0,glad i dot have taks tomorrow ! ! #thankful #s...,glad dot taks tomorrow ! ! #thankful #startho
2,3,-1.0,1-3 vs celtics in the regular season = were fu...,1-3 vs celtics regular season = fucked play pl...
3,4,-1.0,<user> i could actually kill that girl i'm so ...,could actually kill girl i'm sorry ! ! !
4,5,-1.0,<user> <user> <user> i find that very hard to ...,find hard believe im afraid
5,6,-1.0,wish i could be out all night tonight ! <user>,wish could night tonight !
6,7,-1.0,<user> i got kicked out the wgm,got kicked wgm
7,8,-1.0,rt <user> <user> <user> yes she is ! u tell it...,rt yes ! u tell ! lips closed okay
8,9,-1.0,why is she so perfect <url>,why perfect
9,10,-1.0,<user> hi harry ! did u havea good time in aus...,hi harry ! u havea good time aus ? didnt get 2...


In [98]:
combi['clean_tweet'][:100000].to_csv("prep_data_neg.txt", index=False, header=False)
combi['clean_tweet'][100000:200000].to_csv("prep_data_pos.txt", index=False, header=False)
combi['clean_tweet'][200000:].to_csv("prep_data_test.txt", index=False, header=False)